In [1]:
import torch
from torchdrug import datasets, core, models, tasks, utils
from torch import optim,utils

# The following functions takes 8 mins
dataset = datasets.ZINC250k("./molecule-datasets/", kekulize=True,
                            node_feature="symbol")

d:\Anaconda\envs\molecule\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading ./molecule-datasets/250k_rndm_zinc_drugs_clean_3.csv:  50%|█████     | 249456/498911 [00:01<00:01, 141360.92it/s]
Constructing molecules from SMILES:   0%|          | 0/249455 [00:00<?, ?it/s]d:\Anaconda\envs\molecule\lib\site-packages\torchdrug\utils\decorator.py:189: UserWarning: from_molecule(): argument `node_feature` is deprecated in favor of `atom_feature`
  warnings.warn("%s(): argument `%s` is deprecated in favor of `%s`" % (obj.__name__, key, value))
Constructing molecules from SMILES: 100%|██████████| 249455/249455 [03:08<00:00, 1322.51it/s]


In [32]:

subset = dataset
subset.data = dataset.data[:3]


In [30]:
dataset.num_bond_type

3

In [21]:
print(subset.atom_types)

[6, 7, 8, 9, 16, 17, 35, 53]


In [35]:
import torch
from torchdrug import datasets, core, models, tasks, utils
from torch import optim,utils

import pickle
with open("zinc250k.pkl", "wb") as fout:
    pickle.dump(subset, fout)
with open("zinc250k.pkl", "rb") as fin:
    dataset = pickle.load(fin)

In [53]:
print(f"dim:{dataset.node_feature_dim}, num_relation={dataset.num_bond_type}, ")

dim:67, num_relation=3, 


In [31]:
dataset.transform = None
# (1)
model = models.RGCN(input_dim=18,
                    num_relation=3,
                    hidden_dims=[256, 256, 256, 256], batch_norm=False)
# (2)
task = tasks.GCPNGeneration(model, [6, 7, 8, 9, 15, 16, 17, 35, 53], max_edge_unroll=12,
                            max_node=38, criterion="nll")


In [33]:
optimizer = optim.Adam(task.parameters(), lr = 1e-3)
solver = core.Engine(task, subset, None, None, optimizer,
                     gpus=(0,), batch_size=128, log_interval=10)

23:12:33   Preprocess training set
23:12:33   {'batch_size': 128,
 'class': 'core.Engine',
 'gpus': (0,),
 'gradient_interval': 1,
 'log_interval': 10,
 'logger': 'logging',
 'num_worker': 0,
 'optimizer': {'amsgrad': False,
               'betas': (0.9, 0.999),
               'capturable': False,
               'class': 'optim.Adam',
               'differentiable': False,
               'eps': 1e-08,
               'foreach': None,
               'fused': False,
               'lr': 0.001,
               'maximize': False,
               'weight_decay': 0},
 'scheduler': None,
 'task': {'agent_update_interval': 10,
          'atom_types': [6, 7, 8, 9, 15, 16, 17, 35, 53],
          'baseline_momentum': 0.9,
          'class': 'tasks.GCPNGeneration',
          'criterion': 'nll',
          'gamma': 0.9,
          'hidden_dim_mlp': 128,
          'max_edge_unroll': 12,
          'max_node': 38,
          'model': {'activation': 'relu',
                    'batch_norm': False,
         

In [34]:
solver.load("gcpn_zinc250k_5epoch.pkl")
results = task.generate(num_sample=5, max_resample=5)
print(results.to_smiles())

23:12:35   Load checkpoint from gcpn_zinc250k_5epoch.pkl


d:\Anaconda\envs\molecule\lib\site-packages\torch\nn\functional.py:1956: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


['CC(=NC(N)=C(C)C(=O)Cl)C(N)=O', 'CN1N=C(C(=O)O)C=C1N1CCC(O)C1', 'ClC1=CC=C(OCCN2CCNCC2)C=C1', 'CCN=C1SC=C(C)N1C(=O)C1CCNC1', 'CNCC1CN(C2=CC=C(C)C=C2)C(=O)NC2=CC=CC=C2OCCCO1']


In [42]:
import warnings
warnings.filterwarnings("ignore")
import logging
logging.disable(logging.CRITICAL)



def simple_molecule_generation():
    with open("zinc250k.pkl", "rb") as fin:
        dataset = pickle.load(fin)
    dataset.transform = None
    model = models.RGCN(input_dim=18,
                        num_relation=3,
                        hidden_dims=[256, 256, 256, 256], batch_norm=False)
    task = tasks.GCPNGeneration(model, [6, 7, 8, 9, 15, 16, 17, 35, 53], max_edge_unroll=12,
                                max_node=38, criterion="nll")
    optimizer = optim.Adam(task.parameters(), lr = 1e-3)
    solver = core.Engine(task, dataset, None, None, optimizer,
                            gpus=(0,), batch_size=128, log_interval=10)
    solver.load("gcpn_zinc250k_5epoch.pkl")
    results = task.generate(num_sample=1, max_resample=10)
    return results.to_smiles()

In [44]:
simple_molecule_generation()

['O=S(=O)(NC1=CC=CN=C1Cl)C1=CC=CC=C1Cl']

In [57]:
import logging
logging.disable(logging.NOTSET)

In [ ]:
dataset = datasets.ZINC250k("./molecule-datasets/", kekulize=True)

In [2]:
# define the task and load the pretrained model
dataset.transform = None
model = models.RGCN(input_dim=18,
                    num_relation=3,
                    hidden_dims=[256, 256, 256, 256], batch_norm=False)
task = tasks.GCPNGeneration(model, [6, 7, 8, 9, 15, 16, 17, 35, 53], max_edge_unroll=12,
                            max_node=38, task="qed", criterion=("ppo", "nll"),
                            reward_temperature=1,
                            agent_update_interval=3, gamma=0.9)
optimizer = optim.Adam(task.parameters(), lr = 1e-5)
solver = core.Engine(task, dataset, None, None, optimizer,
                     gpus=(0,), batch_size=512, log_interval=10)
solver.load('gcpn_zinc250k_5epoch.pkl', load_optimizer=False)

19:21:54   Preprocess training set


19:21:55   {'batch_size': 512,
 'class': 'core.Engine',
 'gpus': (0,),
 'gradient_interval': 1,
 'log_interval': 10,
 'logger': 'logging',
 'num_worker': 0,
 'optimizer': {'amsgrad': False,
               'betas': (0.9, 0.999),
               'capturable': False,
               'class': 'optim.Adam',
               'differentiable': False,
               'eps': 1e-08,
               'foreach': None,
               'fused': False,
               'lr': 1e-05,
               'maximize': False,
               'weight_decay': 0},
 'scheduler': None,
 'task': {'agent_update_interval': 3,
          'atom_types': [6, 7, 8, 9, 15, 16, 17, 35, 53],
          'baseline_momentum': 0.9,
          'class': 'tasks.GCPNGeneration',
          'criterion': ('ppo', 'nll'),
          'gamma': 0.9,
          'hidden_dim_mlp': 128,
          'max_edge_unroll': 12,
          'max_node': 38,
          'model': {'activation': 'relu',
                    'batch_norm': False,
                    'class': 'models

In [4]:
# RL finetuning
solver.train(num_epoch=1)

19:22:53   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
19:22:53   Epoch 0 begin


d:\Anaconda\envs\molecule\lib\site-packages\torch\nn\functional.py:1956: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


19:23:18   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
19:23:19   PPO objective: 1.81444
19:23:19   QED: 0.621351
19:23:19   QED (max): 0.885289
19:23:19   edge acc: 0.906658
19:23:19   edge loss: 0.221393
19:23:19   node1 acc: 0.647624
19:23:19   node1 loss: 0.896229
19:23:19   node2 acc: 0.781187
19:23:19   node2 loss: 0.61765
19:23:19   stop acc: 0.874163
19:23:19   stop bce loss: 0.208844
19:23:19   total loss: 1.94412
19:23:30   1 / 300 molecules are invalid even after 20 resampling
19:23:32   1 / 135 molecules are invalid even after 20 resampling
19:23:48   1 / 149 molecules are invalid even after 20 resampling
19:23:56   2 / 512 molecules are invalid even after 20 resampling
19:23:58   1 / 510 molecules are invalid even after 20 resampling
19:24:12   1 / 512 molecules are invalid even after 20 resampling
19:24:32   1 / 512 molecules are invalid even after 20 resampling
19:25:16   1 / 512 molecules are invalid even after 20 resampling
19:25:48   1 / 512 molecules are invalid even after 20 res

KeyboardInterrupt: 

In [5]:
solver.save("gcpn_zinc250k_qed_rl_1epoch.pkl")

20:51:10   Save checkpoint to gcpn_zinc250k_qed_rl_1epoch.pkl


In [84]:
results = task.generate(num_sample=1, max_resample=5).cpu()
results.visualize()